# Databases SQL server load

This notebook allows to upload the downloaded and preprocessed databases into a mySQL server. To use this code, the `local_infile` variable on the mySQL server needs to be set to True. Execute the following command on the mySQL server (or mySQL Workbench) with a priviledged user:
```sql
SET GLOBAL local_infile = true;
```
This will allow to upload the databases from local file, which greatly speedups the uploading process.

In [ ]:
from cpiextract.sql_connection import connect_to_mysql

Update SQL config with the required data. Do not remove or modify the `allow_local_infile` setting.

In [ ]:
sql_config = {
    "host": "XXX.XX.XX.XXX",
    "user": "user",
    "password": "password",
    "database": "cpie",
    "allow_local_infile": True
}

cnx = connect_to_mysql(sql_config, attempts=3)

assert cnx and cnx.is_connected()

Load the dbs_config file. \
Replace the data folder with the appropriate path.

In [ ]:
data_path = 'data/'

In [ ]:
import json
import os

json_file_path = os.path.join(data_path, 'dbs_config.json')

# Load the JSON data
with open(json_file_path, 'r') as f:
    dbs_config = json.load(f)

### Functions to create the SQL table and load the database into it

In [ ]:
def create_table(cursor, table_name, headers, header_types):
    # Define column types based on your data
    column_types = ['INT AUTO_INCREMENT PRIMARY KEY'] + [header_types[h] for h in headers]
    columns = ', '.join([f"`{header}` {col_type}" for header, col_type in zip(['id'] + headers, column_types)])
    # column_types = [header_types[h] for h in headers]
    # columns = ', '.join([f"{header} {col_type}" for header, col_type in zip(headers, column_types)])
    query = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(query)

    query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})"
    # Execute the CREATE TABLE query
    cursor.execute(query)
    print('Table created')

In [ ]:
def load_csv_to_table(cnx, config):
    cursor = cnx.cursor()

    file_path = 'data/'+ str(config["file"])
    table_name = config["name"]
    sep = ',' if config["file"].endswith(".csv") else '\t'

    # Open the TSV file and create the table if it doesn't exist
    with open(file_path, 'r') as file:
        headers = file.readline().strip().split(sep)
        # Create the table
        create_table(cursor, table_name, headers, config["columns"])  

        # Prepare the INSERT query  
        query = f"""
            LOAD DATA LOCAL INFILE '{file_path}' INTO TABLE {table_name}
            FIELDS TERMINATED BY '{sep}'
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            IGNORE 1 ROWS
            ({','.join([f"`{header}`" for header in headers])});
        """
        
        cursor.execute(query)
        # Commit changes and close the cursor and connection
        cnx.commit()
        print("Database loaded")
        cursor.close()


## BindingDB

In [ ]:
config = dbs_config["BindingDB"]

In [ ]:
load_csv_to_table(cnx, config)

## ChEMBL

In [ ]:
config = dbs_config["ChEMBL"]

In [ ]:
load_csv_to_table(cnx, config)

## CTD

In [ ]:
config = dbs_config["CTD"]

In [ ]:
load_csv_to_table(cnx, config)

## Drugbank

In [ ]:
config = dbs_config["DrugBank"]

In [ ]:
load_csv_to_table(cnx, config)

## DrugCentral

In [ ]:
config = dbs_config["DrugCentral"]

In [ ]:
load_csv_to_table(cnx, config)

## DTC

In [ ]:
config = dbs_config["DTC"]

In [ ]:
load_csv_to_table(cnx, config)

## STITCH

In [ ]:
config = dbs_config["STITCH"]

In [ ]:
load_csv_to_table(cnx, config)